# Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import warnings
warnings.filterwarnings('ignore')

# Import required libraries

In [3]:
import os
import requests
import json
import pandas as pd
from datetime import datetime

# Setting up API and user information

In [10]:
user_key = 'Input your User Key'
data_apikey = "Input your Data API Key"

root_url = f'https://zipline.fin.cloud.ainode.ai/{user_key}/'

# health check

In [11]:
requests.get(root_url).json()

# upload to backtest server

In [ ]:
# Upload strategy file
endpoint = 'upload/strategy/'
url = root_url + endpoint

file_base_path = "/content/drive/MyDrive/~~~" # Input a path that contains a proven your strategy
strategy_nm = "Input your Strategy file" # example : "Momentum.py" (python file)
file_path = file_base_path + strategy_nm

with open(file_path, "rb") as f:
    files = {"file": f}
    response = requests.post(url, files=files)

print("📂 Upload Response:", response.json())

In [ ]:
# Check strategy file upload
endpoint = 'upload/strategy/check/'
url = root_url + endpoint

params = {"strategy_name": "Input your strategy name"} # example : "Momentum.py" --> "Momentum"

response = requests.get(url, params=params)
print(response.json()['content'])

# delete

In [ ]:
# # Delete uploaded strategy settings file
# endpoint = 'upload/strategy/delete/'
# url = root_url + endpoint

# params = {"strategy_name": "Input your strategy name"} # example : "Momentum.py" --> "Momentum"

# # Sending a DELETE request
# response = requests.delete(url, params=params)
# print(response.json())

# config


In [ ]:
# --- 2. Defining backtest execution parameters ---

strategy_file_name = "Input your strategy file name" # example : "Momentum.py"

strategy_config_params = {
  "rebalancing_config": {             # Rebalancing settings
    "rebalancing_interval_hours": 72, ## Rebalancing cycle (choose between 6, 12, 24, and 72 hours)
    "minimum_candidates": 0           ## Minimum number of symbols to select
  },
  "strategy_config": {                # Setting strategy parameters
    "Input your parameter name": "Input your parameter value",

    "Input your parameter name": "Input your parameter value"
  }
}

start_date_str = "2025-01-01"
end_date_str = "2025-03-01"
lookback_min = 180 # Max lookback minutes the script needs for data history
initial_capital = 200000
leverage = 10
symbols = ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'BCHUSDT', 'LTCUSDT',
           'ADAUSDT', 'ETCUSDT', 'TRXUSDT', 'DOTUSDT', 'DOGEUSDT', 'SOLUSDT',
           'BNBUSDT', 'ICPUSDT', 'FILUSDT', 'XLMUSDT', 'ONTUSDT', 'QTUMUSDT',
           'NKNUSDT', 'AVAXUSDT', 'CELOUSDT', 'WAXPUSDT', 'DYMUSDT', 'APTUSDT',
           'FLOWUSDT', 'GTCUSDT', 'SEIUSDT', 'ATOMUSDT', 'NEARUSDT', 'STXUSDT',
           'MINAUSDT', 'BSVUSDT', 'EGLDUSDT', 'RVNUSDT', 'ONEUSDT', 'NEOUSDT',
           'JUPUSDT', 'ZILUSDT', 'XTZUSDT', 'LUNCUSDT', 'CKBUSDT', 'IOTAUSDT',
           'THETAUSDT', 'ICXUSDT', 'ALGOUSDT', 'LSKUSDT', 'CFXUSDT', 'TONUSDT',
           'MEMEUSDT', 'SXPUSDT', 'KASUSDT', 'HBARUSDT', 'IOSTUSDT', 'BEAMUSDT',
           'FETUSDT', 'XVGUSDT', 'SUIUSDT', 'VETUSDT', 'KSMUSDT', 'ARBUSDT',
           'ARUSDT', 'RUNEUSDT', 'IOTXUSDT', 'TAIKOUSDT', 'COREUSDT', 'BBUSDT',
           'COTIUSDT', 'NTRNUSDT'] # Select Symbol for Backtest
calendar = "24/7" # fixed variable
frequency = "minute" # fixed variable
weight = 0.05

generate_report_flag = True # True : backtest logging + html backtest result
                            # False : backtest logging

# run backtest

In [ ]:
# Just run this shell
# --- 3. Configuring API Request Body (Payload) ---
request_payload = {
    "data_apikey": data_apikey,
    "strategy": strategy_file_name,
    "strategy_config": strategy_config_params,
    "start_date": start_date_str,
    "end_date": end_date_str,
    "lookback_minutes": lookback_min,
    "capital": initial_capital,
    "leverage": leverage,
    "symbols": symbols,
    "calendar": calendar,
    "frequency": frequency,
    "weight": weight,
    "generate_pyfolio_report": generate_report_flag
}

# --- 4. Configure API endpoint URL ---
endpoint = 'run/backtest/'
url = root_url + endpoint

# --- 5. Executing API calls and processing results ---
try:
    response = requests.post(url, json=request_payload)
    response.raise_for_status()
    print(f"\n--- Backtest execution successful (Status Code: {response.status_code}) ---")

    try:
        result_data = response.json()
        report_type = result_data.get('report_type') # Check report type

        if report_type == 'html':
            print("Report Type: HTML Report included.")
            html_content = result_data.get('html_content')
            logs = result_data.get('logs', 'No stderr logs received.')
            stdout_logs = result_data.get('stdout', 'No stdout received.')

            if html_content:
                path = "/content/drive/MyDrive/backtest_report" # Modify route

                if not os.path.exists(path):
                  os.makedirs(path)

                report_filename = os.path.join(path, f"{datetime.now().strftime('%Y-%m-%d %H:%M')}_backtest_report.html")
                try:
                    with open(report_filename, "w", encoding="utf-8") as f:
                        f.write(html_content)
                    print(f"HTML report received and saved successfully as '{report_filename}'.")
                except Exception as e:
                    print(f"ERROR: Failed to save received HTML report: {e}")
                    print("\n--- Received HTML Content (Snippet) ---")
                    print(html_content[:1000] + "...") # Output some content when saving fails
            else:
                print("WARN: Report type was 'html' but no HTML content found in response.")

            # print logging
            print("\n--- Execution Logs (stderr) ---")
            print(logs)
            if stdout_logs:
                print("\n--- Execution Output (stdout) ---")
                print(stdout_logs)

        elif report_type == 'logs_only':
            print(f"Report Type: Logs Only (Report generation skipped).")
            print(f"Message: {result_data.get('message')}")
            print("\n--- Execution Logs (stderr) ---")
            print(result_data.get('logs', 'No stderr logs received.'))
            if 'stdout' in result_data:
                 print("\n--- Execution Output (stdout) ---")
                 print(result_data.get('stdout', 'No stdout received.'))
        else:
            print(f"WARN: Received successful response with unknown report_type: '{report_type}'")
            print("\n--- Full JSON Response ---")
            print(json.dumps(result_data, indent=2, ensure_ascii=False))

    except json.JSONDecodeError:
        print("ERROR: Failed to decode JSON response from successful API call.")
        print(f"Content-Type: {response.headers.get('content-type', 'N/A')}")
        print("\n--- Received Raw Content (First 1000 chars) ---")
        print(response.text[:1000] + "...")

except requests.exceptions.Timeout:
    print(f"\n--- API call failed: Timeout) ---")
except requests.exceptions.HTTPError as e:
    print(f"\n--- API call failed: HTTP Error {e.response.status_code} ---")
    try:
        error_details = e.response.json()
        print("Error details:")
        print(json.dumps(error_details, indent=2, ensure_ascii=False))
    except json.JSONDecodeError:
        print("Error response content (Non-JSON):")
        print(e.response.text)
except requests.exceptions.RequestException as e:
    print(f"\n--- API call failure: Request Error ---")
    print(f"Error connecting to or requesting the API server ({url}): {e}")
except Exception as e:
    print(f"\n--- Unexpected error occurred ---")
    print(f"Error type: {type(e).__name__}, Content: {e}")